In [ ]:
from __future__ import annotations
import logging
from typing import Optional
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL
from datetime import datetime
from sqlalchemy import inspect 
import os
from dotenv import load_dotenv
from venv import logger

In [ ]:
class DBConnector:
    SUPPORTED_DB_TYPES = {
        'postgresql+psycopg2': 'PostgreSQL',
    }

    DEFAULT_POOL_CONFIG = {
        'pool_size': 5,
        'max_overflow': 10,
        'pool_timeout': 30,
        'pool_recycle': 3600
    }

    MAX_RETRIES = 3
    RETRY_DELAY = 2 

    host: str
    port: int
    user_name: str
    password: str
    dbname: str
    engine_name: str
    engine: Optional[Engine]

    def __init__(self, host: str, port: int, user_name: str, password: str, dbname: str, engine_name: str):
        self.host = host
        self.port = int(port)
        self.user_name = user_name
        self.password = password
        self.dbname = dbname
        self.engine_name = engine_name
        self.engine = None
        self._last_connection_time: Optional[datetime] = None

    def validate_credentials(self) -> bool:
        if not isinstance(self.host, str) or not self.host.strip():
            raise ValueError("host должен быть непустой строкой.")
        if not isinstance(self.port, int) or not (1 <= self.port <= 65535):
            raise ValueError("port должен быть целым числом в диапазоне 1-65535.")
        if not isinstance(self.user_name, str) or not self.user_name.strip():
            raise ValueError("username должен быть непустой строкой.")
        if not isinstance(self.dbname, str) or not self.dbname.strip():
            raise ValueError("dbname должен быть непустой строкой.")
        if self.engine_name not in self.SUPPORTED_DB_TYPES:
            raise ValueError(f"engine_name '{self.engine_name}' не поддерживается. Поддержаны: {list(self.SUPPORTED_DB_TYPES.keys())}")
        return True

    def set_connection(self) -> bool:
        self.validate_credentials()

        connection_data = URL.create(
            drivername=self.engine_name,
            username=self.user_name,
            password=self.password,
            host=self.host,
            port=self.port,
            database=self.dbname
        )

        pool_cfg = self.DEFAULT_POOL_CONFIG.copy()

        try:
            engine = create_engine(
                connection_data,
                pool_size=pool_cfg['pool_size'],
                max_overflow=pool_cfg['max_overflow'],
                pool_timeout=pool_cfg['pool_timeout'],
                echo=False,
                pool_recycle=pool_cfg['pool_recycle']
            )

            with engine.connect() as db_connection:
                init_query_result = db_connection.execute(text('SELECT 1;'))
                self.engine = engine
                self._last_connection_time = datetime.utcnow()
                return True

        except SQLAlchemyError as err:
            logger.error(f'Connection error! {err}')
            raise ConnectionError(f"Не удалось подключиться к БД: {err}")

    def disconnect(self) -> None:
        try:
            if self.engine is not None:
                self.engine.dispose()
        finally:
            self.engine = None
            self._last_connection_time = None


    def get_engine(self) -> Engine:
        if self.engine is None:
            raise RuntimeError("Нет активного подключения. Вызовите set_connection().")
        return self.engine


In [ ]:
from typing import Dict


class UserDBManager:
    def __init__(self, 
                 db_connection: DBConnector,
                 schema: str = "public",
                 config = None):
        if not isinstance(db_connection, DBConnector):
            raise ValueError("db_connection должен быть объектом DBConnector")
        
        
        self.db_connection: DBConnector = db_connection
        self.engine: Engine = db_connection.get_engine()
        self.schema: str = schema
        self.inspector = inspect(self.engine)
        self.view_counter = 0
        
        self.config = {
            'max_depth': 10,
            'batch_size': 1000,
        }
        if config:
            self.config.update(config)

    def get_composite_keys_ratio(self) -> float:
        table_names = self.inspector.get_table_names(schema=self.schema)
        if not table_names:
            return 0.0
            
        composite_pk_count = 0
        tables_with_pk_count = 0
        
        for table in table_names:
            pk_constraint = self.inspector.get_pk_constraint(table, schema=self.schema)
            if pk_constraint and pk_constraint['constrained_columns']:
                tables_with_pk_count += 1
                if len(pk_constraint['constrained_columns']) > 1:
                    composite_pk_count += 1
                    
        return composite_pk_count / tables_with_pk_count if tables_with_pk_count > 0 else 0.0

    def get_non_integer_keys_ratio(self) -> float:
        table_names = self.inspector.get_table_names(schema=self.schema)
        if not table_names:
            return 0.0
            
        non_int_pk_count = 0
        tables_with_pk_count = 0

        integer_types = ('INTEGER', 'BIGINT', 'SMALLINT', 'SERIAL', 'BIGSERIAL')

        for table in table_names:
            pk_constraint = self.inspector.get_pk_constraint(table, schema=self.schema)
            cols = pk_constraint.get('constrained_columns', [])
            
            if cols:
                tables_with_pk_count += 1
                is_non_int = False
                
                columns_info = self.inspector.get_columns(table, schema=self.schema)
                col_type_map = {col['name']: str(col['type']).upper() for col in columns_info}
                
                for pk_col in cols:
                    col_type = col_type_map.get(pk_col, '')
                    # Проверяем, содержит ли тип подстроку int (грубая проверка, но обычно работает для SQLALchemy типов)
                    # Более точная проверка - через isinstance(col['type'], sqlalchemy.types.Integer)
                    if not any(t in col_type for t in integer_types):
                        is_non_int = True
                        break
                
                if is_non_int:
                    non_int_pk_count += 1
                    
        return non_int_pk_count / tables_with_pk_count if tables_with_pk_count > 0 else 0.0

    def get_char_attributes_memory_waste_ratio(self) -> float:
        table_names = self.inspector.get_table_names(schema=self.schema)
        total_char_columns = 0
        wasteful_char_columns = 0
        
        for table in table_names:
            columns = self.inspector.get_columns(table, schema=self.schema)
            for col in columns:
                col_type_str = str(col['type']).upper()
                
                # Проверяем, является ли это CHAR (обычно CHAR или CHARACTER)
                # Важно: VARCHAR обычно отображается как VARCHAR
                if col_type_str.startswith('CHAR') or col_type_str.startswith('CHARACTER'):
                    # Исключаем 'CHARACTER VARYING' (это VARCHAR)
                    if 'VARYING' not in col_type_str:
                        total_char_columns += 1
                        # Получаем длину. SQLAlchemy типы хранят длину в атрибуте length
                        length = getattr(col['type'], 'length', None)
                        
                        # Если длина не указана явно, часто по стандарту 1, но лучше проверить.
                        # Если длина > 1, считаем это потенциальной растратой (лучше VARCHAR)
                        if length is not None and length > 1:
                            wasteful_char_columns += 1
                            
        return wasteful_char_columns / total_char_columns if total_char_columns > 0 else 0.0

    def get_tables_without_on_update_ratio(self) -> float:
        sql = text("""
            SELECT 
                COUNT(*) as total_fks,
                SUM(CASE WHEN update_rule IN ('NO ACTION', 'RESTRICT') THEN 1 ELSE 0 END) as no_update_action
            FROM information_schema.referential_constraints
            WHERE constraint_schema = :schema
        """)
        
        with self.engine.connect() as conn:
            result = conn.execute(sql, {"schema": self.schema}).fetchone()
            
        if not result or result.total_fks == 0:
            return 0.0
            
        return result.no_update_action / result.total_fks

    def get_tables_without_on_delete_ratio(self) -> float:
        sql = text("""
            SELECT 
                COUNT(*) as total_fks,
                SUM(CASE WHEN delete_rule IN ('NO ACTION', 'RESTRICT') THEN 1 ELSE 0 END) as no_delete_action
            FROM information_schema.referential_constraints
            WHERE constraint_schema = :schema
        """)

        with self.engine.connect() as conn:
            result = conn.execute(sql, {"schema": self.schema}).fetchone()
            
        if not result or result.total_fks == 0:
            return 0.0
            
        return result.no_delete_action / result.total_fks

    def get_all_metrics(self) -> Dict[str, float]:
        return {
            "доля_составных_ключей": self.get_composite_keys_ratio(),
            "доля_не_целочисленных_ключей": self.get_non_integer_keys_ratio(),
            "доля_char_атрибутов_с_растратой_пямяти": self.get_char_attributes_memory_waste_ratio(),
            "доля_таблиц_без_on_update": self.get_tables_without_on_update_ratio(),
            "доля_таблиц_без_on_delete": self.get_tables_without_on_delete_ratio(),
            # "доля_неатомарных_атрибутов": self.get_non_atomic_attributes_ratio() 
        }



In [70]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user_name = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
dbname = os.getenv('DBNAME')
engine_name = 'postgresql+psycopg2'

In [ ]:
cur_schema = "public"

In [ ]:
conn = DBConnector(host, port, user_name, password, dbname, engine_name)
db_manager = UserDBManager(db_connection=conn, schema=cur_schema)
conn.set_connection()

/var/folders/0n/5qlwvd9d02n8kssx435yjb9c0000gp/T/ipykernel_5835/3138801911.py:74: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self._last_connection_time = datetime.utcnow()


True

In [74]:
db_metrics = db_manager.get_all_metrics()
db_metrics

{'доля_составных_ключей': 0.0,
 'доля_не_целочисленных_ключей': 0.0,
 'доля_char_атрибутов_с_растратой_пямяти': 0.0,
 'доля_таблиц_без_on_update': 1.0,
 'доля_таблиц_без_on_delete': 1.0}